# EXPECTING THE UNEXPECTED

Systems built with software can be fragile. We need to have a way to work around the spectrum of failures that plague computer systems.

We will study **exceptions**, special error objects raised when a normal response is impossible.



# RAISING EXCEPTIONS

Python's normal behaviour is to execute statements in the order they are found.

A few statements, specifically `if`, `while`, and `for`, alter the simple top-to-bottom sequence of statement execution.

Additionally, an exception can break the sequential flow of execution.

Exceptions are raised, and this interrupts the sequential execution of statements.

In Python, the exception that's raised is also an object. There are many different exception classes available and we can easily define more of our own.

The one thing that all exceptions have in common is that they are instances of a class that inherits from the `BaseException` class.

When an exception is raised, everything that was supposed to happen is pre-empted.

Instead, exception handling replaces the normal processing.

The easiest way to cause an exception to occur is to do something silly. 

For example, any time Python encounters a line in your program that it can't understand, it bails with SyntaxError, which is a type of exception.



In [1]:
print "Hello, world!"

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (2804745302.py, line 1)

The print() function requires the arguments to be enclosed in parentheses. So, if we type the preceding command into a Python 3 interpreter, we raise a SyntaxError exception.

We can partition exceptions into roughly four categories. Some cases are blurry, but some edges have a bright line separating them:

Sometimes, exceptions are indicators of something clearly wrong in our program. Exceptions like SyntaxError and NameError mean we need to find the indicated line number and fix the problem.


Sometimes, exceptions are indicators of something wrong in the Python runtime. There's a RuntimeError exception that can get raised. In many cases, this is resolved by downloading and installing a newer Python. 

Some exceptions are design problems. We may fail to account for an edge case properly and sometimes try to compute an average of an empty list. This will result in a `ZeroDivisionError`. When we find these, again, we'll have to go to the indicated line number. But once we've found the resulting exception, we'll need to work backwards from there to find out what caused the problem that raised the exception. 

# EFFECTS OF EXCEPTIONS

When an exception is raised, it appears to stop the program execution immediately.

Any lines that were supposed to run after the exception is raised are not executed unless the exception is handled by an `except` clause, the program will exit with an error message.

We can control how exceptions propagate from the initial `raise` statement. We can react to and deal with the exception inside either of these methods in the call stack.

# HANDLING EXCEPTIONS

If we encounter an exception situation, how should our code react to or recover from it? 

We handle exceptions by wrapping any code that might throw one (whether it is exception code itself, or a call to any function or method that may have an exception raised inside it) inside a `try...except` clause. 

The most basic syntax looks like this:

In [2]:
from typing import NoReturn
def never_returns() -> NoReturn:
    print("I am about to raise an exception")
    raise Exception("This is always raised")
    print("This line will never execute")
    return "I won't be returned"

In [3]:
never_returns()

I am about to raise an exception


Exception: This is always raised

In [4]:
def handler() -> None:
    try:
        never_returns()
        print("Never executed")
    except Exception as ex:
        print(f"I caught an exception: {ex!r}")
    print("Executed after the exception")



In [5]:
handler()

I am about to raise an exception
I caught an exception: Exception('This is always raised')
Executed after the exception


The `never_returns()` function happily informs us that it is about to raise an exception and raises it. 

The `handler()` function's except clause catches the exception. Once caught, we are able to clean up after ourselves (in this case, by outputting that we are handling the situation), and continue on our way. 

The remainder of the code in the `never_returns()` function remains unexecuted, but the code in the `handler()` function after the try: statement is able to recover and continue.

Note the indentation around `try and except`. The `try` clause wraps any code that might throw an exception. The `except` clause is then back on the same indentation level as the `try` line. Any code to handle the exception is indented inside the `except` clause. Then normal code resumes at the original indentation level.

The problem with the preceding code is that it uses the `Exception` class to match any type of exception. 

What if we were writing some code that could raise either `TypeError` or `ZeroDivisionError`? 

We might need to catch `ZeroDivisionError` because it reflects a known object state, but let any other exceptions propagate to the console because they reflect bugs we need to catch and kill. 

Can you guess the syntax?

In [6]:
from typing import Union

def funny_division(divisor:float) -> Union[str, float]:
    try:
        return 100 / divisor
    except ZeroDivisionError:
        return "Zero is not a good idea!"


This function does a simple computation. We've provided the type hint of float for the divisor parameter. We can provide an integer, and ordinary Python type coercion will work. 

The mypy tool is aware of the ways integers can be coerced to floats, saving it from having to obsess over the parameter types.

We do, however, have to be very clear about the return types. 

If we don't raise an exception, we'll compute and return a floating result. If we do raise a `ZeroDivisionError` exception, it will be handled, and we'll return a string result. 

Any other exceptions? Let's try it and see:

In [7]:
funny_division(0)

'Zero is not a good idea!'

In [8]:
print(funny_division(50.0))

2.0


In [9]:
print(funny_division("hello"))

TypeError: unsupported operand type(s) for /: 'int' and 'str'

 If we don't specify matching the `ZeroDivisionError` exception class, our handler would also see the `TypeError`, and accuse us of dividing by zero when we sent it a string, which is not a proper behavior at all.
 
Python also has a bare except syntax. Using `except`: with no exception class to match is widely frowned upon because it will prevent an application from simply crashing when it should. 

We generally use `except Exception`: to explicitly catch a sensible set of exceptions.

The bare except syntax is actually the same as using `except BaseException`:, which attempts to handle system-level exceptions that are often impossible to recover from. 

Indeed, this can make it impossible to crash your application when it's misbehaving.

We can even catch two or more different exceptions and handle them with the same code. 

Here's an example that raises three different types of exceptions. 

It handles TypeError and ZeroDivisionError with the same exception handler, but it may also raise a ValueError error if you supply the number 13:

In [10]:
def funny_division2(divisor:float) -> Union[str, float]:
    try:
        if divisor == 13:
            raise ValueError("13 is an unlucky number")
        return 100 / divisor
    except (ZeroDivisionError, TypeError):
        return "Enter a number other than zero"

We've included multiple exception classes in the except clause. This lets us handle a variety of conditions with a common handler. 

Here's how we can test this with a bunch of different values:

In [11]:
for val in (0, "hello", 50.0, 13):
    print(f"Testing {val!r}:", end=" ")
    print(f"Result: {funny_division2(val)}")

Testing 0: Result: Enter a number other than zero
Testing 'hello': Result: Enter a number other than zero
Testing 50.0: Result: 2.0
Testing 13: 

ValueError: 13 is an unlucky number

This is all well and good, but what if we want to catch different exceptions and do different things with them? 

Or maybe we want to do something with an exception and then allow it to continue to bubble up to the parent function, as if it had never been caught?

We do not need any new syntax to deal with these cases. It is possible to stack the `except` clauses, and only the first match will be executed.

For the second question, the `raise` keyword, with no arguments, will re-raise the last exception if we are already inside an exception handler.

In [13]:
def funniest_division(divisor: int) -> Union[str, float]:
    try:
        if divisor == 13:
            raise ValueError("13 is an unlucky number")
        return 100 / divisor
    except ZeroDivisionError:
        return "Enter a number other than zero"
    except TypeError:
        return "Enter a numerical value"
    except ValueError:
        print("No, No, not 13!")
        raise


The last line re-raises the ValueError error, so after outputting No, No, not 13!, it will raise the exception again; we'll still get the original stack trace on the console. 

If we stack exception clauses like we did in the preceding example, only the first matching clause will be run, even if more than one of them fits. 

How can more than one clause match? 

Remember that exceptions are objects, and can therefore be subclassed. 

As we'll see in the next section, most exceptions extend the `Exception` class (which is itself derived from `BaseException`). 

If we have an `except` clause to match `Exception` before we match `TypeError`, then only the `Exception` handler will be executed, because `TypeError` is an `Exception` by inheritance.

This can come in handy in cases where we want to handle some exceptions specifically, and then handle all remaining exceptions as a more general case. 

We can list `Exception` in its own clause after catching all the specific exceptions and handle the general case there.

Often, when we catch an exception, we need a reference to the `Exception` object itself.
 
This most often happens when we define our own exceptions with custom arguments, but can also be relevant with standard exceptions. 

Most exception classes accept a set of arguments in their constructor, and we might want to access those attributes in the exception handler. 

If we define our own Exception class, we can even call custom methods on it when we catch it. 

The syntax for capturing an exception as a variable uses the as keyword:

In [15]:
try :
    raise ValueError("This is an argument")
except ValueError as e:
    print(f"The exception arguments were", e.args)

The exception arguments were ('This is an argument',)


When we run this code, it prints out the string argument that we passed into `ValueError` upon initialization.

We've seen several variations on the syntax for handling exceptions, but we still don't know how to execute code regardless of whether or not an exception has occurred. 

We also can't specify code that should be executed only if an exception does not occur. 

Two more keywords, `finally` and `else`, provide some additional execution paths. Neither one takes any extra arguments.

We'll show an example with the `finally` clause. 

For the most part, we often use context managers instead of exception blocks as a cleaner way to implement a finalization that occurs whether or not an exception interrupted processing. 

The idea is to encapsulate responsibility for finalization in the context manager.

The following example iterates through a number of exception classes, raising an instance of each. 

Then some not-so-complicated exception handling code runs that illustrates the newly introduced syntax:

In [16]:
some_exceptions = [ValueError, TypeError, IndexError, None]
for choice in some_exceptions:
    try:
        print(f"\nRaising {choice}")
        if choice:
            raise choice("An error")
        else:
            print("no exception raised")
    except ValueError:
        print("Caught a ValueError")
    except TypeError:
        print("Caught a TypeError")
    except Exception as e:
        print(f"Caught some other error: {e.__class__.__name__}")
    else:
        print("This code called if there is no exception")
    finally:
        print("This cleanup code is always called")



Raising <class 'ValueError'>
Caught a ValueError
This cleanup code is always called

Raising <class 'TypeError'>
Caught a TypeError
This cleanup code is always called

Raising <class 'IndexError'>
Caught some other error: IndexError
This cleanup code is always called

Raising None
no exception raised
This code called if there is no exception
This cleanup code is always called


Note how the `print` statement in the `finally` clause is executed no matter what happens. 

This is one way to perform certain tasks after our code has finished running (even if an exception has occurred). 

Some common examples include the following:

- Cleaning up an open database connection
- Closing an open file
- Sending a closing handshake over the network

While obscure, the `finally` clause is executed after the `return` statement inside a `try` clause. 

While this can be exploited for post-return processing, it can also be confusing to folks reading the code.

Also, pay attention to the output when no exception is raised: both the `else` and the `finally` clauses are executed. 

The `else` clause may seem redundant, as the code that should be executed when no exception is raised could just be placed after the entire `try...except` block. 

The difference is that the else block will not be executed if an exception is caught and handled. 

We'll see more on this when we discuss using exceptions as flow control later.

Any of the `except`, `else`, and `finally` clauses can be omitted after a `try` block (although else by itself is invalid). 

If you include more than one, the except clauses must come first, then the else clause, with the finally clause at the end. 

You must be sure the order of the except clauses has classes that move from the most specific subclasses to most generic superclasses.

# EXCEPTION HIERARCHY

Most exceptions are subclasses of the `Exception` class. But this is not true of all exceptions. 

The `Exception` class actually extends a class called `BaseException`. 

In fact, all exceptions must extend the `BaseException` class or one of its subclasses.

There are two key built-in exception classes, SystemExit and KeyboardInterrupt, that derive directly from the BaseException class instead of the Exception class. 

The SystemExit exception is raised whenever the program exits naturally, typically because we called the sys.exit() function somewhere in our code (for example, when the user selected an exit menu item, clicked the Close button on a window, entered a command to shut down a server, or the OS sent a signal to the application to terminate). 

This exception is designed to allow us to clean up code before the program ultimately exits.

If we do handle the SystemExit exception, we would normally re-raise the exception, since catching it could stop the program from exiting. 

Imagine a web service with a bug that is holding database locks and can't be stopped without rebooting the server.

We don't want a SystemExit exception to be accidentally caught in generic except Exception: clauses. This is why it derives directly from BaseException.

The KeyboardInterrupt exception is common in command-line programs. It is thrown when the user explicitly interrupts program execution with an OS-dependent key combination (normally, Ctrl + C).

This is a standard way for the user to deliberately interrupt a running program and, like the SystemExit exception, it should almost always respond by terminating the program. 

Also, like SystemExit, it can handle any cleanup tasks inside the finally blocks.

- BaseException:
    - SystemExit
    - KeyboardInterrupt
    - Exception
        - Other Exceptions   


When we use the except: clause without specifying any type of exception, it will catch all subclasses of BaseException; which is to say, it will catch all exceptions, including the two special ones. 

Since we almost always want these to get special treatment, it is unwise to use the except: statement without arguments. 

If you want to catch all exceptions (other than SystemExit and KeyboardInterrupt), always explicitly catch Exception. 

Most Python developers assume that except: without a type is an error and will flag it in code review.

# DEFINING CUSTOM EXCEPTIONS

Occasionally, when we want to raise an exception, we find that none of the built-in exceptions are suitable. 

The distinction is often focused on how applications must handle the exception; when we introduce a new exception it must be because there will be distinct processing in a handler.

There's no good reason to define an exception that's handled exactly like ValueError; we can use ValueError. 

Luckily, it's trivial to define new exceptions of our own. 

The name of the class is usually designed to communicate what went wrong, and we can provide arbitrary arguments in the initializer to include additional information.

All we have to do is inherit from the Exception class or one of the existing exceptions that's semantically similar. 
We don't even have to add any content to the class! 

We can, of course, extend BaseException directly, but this means we're inventing new ways of stopping a running program, a very unusual thing to be creating.

Here's a simple exception we might use in a banking application:

In [17]:
class InvalidWithdrawal(Exception):
    pass

In [18]:
raise InvalidWithdrawal("You don't have $50 in your account")

InvalidWithdrawal: You don't have $50 in your account

The `raise` statement illustrates how to raise the newly defined exception. 

We are able to pass an arbitrary number of arguments into the exception. 

Often a string message is used, but any object that might be useful in a later exception handler can be stored. 

`The Exception.__init__()` method is designed to accept any arguments and store them as a tuple in an attribute named `args`. 

This makes exceptions easier to define without needing to override `__init__()`.

Of course, if we do want to customize the initializer, we are free to do so. Here's a revision to the above exception whose initializer accepts the current balance and the amount the user wants to withdraw. In addition, it adds a method to calculate how overdrawn the request is:


In [19]:
from decimal import Decimal

class InvalidWithdrawal(ValueError): 
    def __init__(self, balance: Decimal, amount: Decimal) -> None: 
        super().__init__(f"account doesn't have ${amount}") 
        self.amount = amount 
        self.balance = balance 
    def overage(self) -> Decimal: 
        return self.amount - self.balance 


Since we're working with currency, we've imported the Decimal class of numbers. 

We can't use Python's default int or float types for money where there are a fixed number of decimal places and exquisitely complex rounding rules that assume exact decimal arithmetic.

(Also note that the account number is not part of the exception. Bankers frown on using account numbers in a way that could expose them in a log or a traceback message.)

Here's an example of creating an instance of this exception:

In [20]:
raise InvalidWithdrawal(Decimal('25.00'), Decimal('50.00'))

InvalidWithdrawal: account doesn't have $50.00

In [21]:
try:
    balance = Decimal('25.00')
    raise InvalidWithdrawal(balance, Decimal('50.00')) 
except InvalidWithdrawal as ex: 
    print("I'm sorry, but your withdrawal is " 
            "more than your balance by " 
            f"${ex.overage()}") 


I'm sorry, but your withdrawal is more than your balance by $25.00


Here we see a valid use of the as keyword to save the exception in a local variable, `ex`. 

By convention, most Python coders assign the exception a variable like `ex`, `exc`, or `exception`; although, as usual, you are free to call it `the_exception_raised_above`, or `aunt_sally` if you prefer.

There are many reasons for defining our own exceptions. 

It is often useful to add information to the exception or log it in some way. 

But the utility of custom exceptions truly comes to light when creating a framework, library, or API that is intended for access by other programmers. 

In that case, be careful to ensure your code is raising exceptions that make sense to the client programmer. Here are some criteria:

- They should clearly describe what went on. The KeyError exception, for example, provides the key that could not be found.

- The client programmer should easily see how to fix the error (if it reflects a bug in their code) or handle the exception (if it's a situation they need to be made aware of).

- The handling should be distinct from other exceptions. If the handling is the same as an existing exception, reusing the existing exception is best.


Now that we've looked at raising exceptions and defining new exceptions, we can look at some of the design considerations that surround exceptional data and responding to problems. 

There are a number of alternative design choices, and we'll start with the idea that exceptions, in Python, can be used for a number of things that aren't – strictly speaking – erroneous.

# EXCEPTIONS ARE NOT EXCEPTIONAL

Novice programmers tend to think of exceptions as only useful for exceptional circumstances. 

However, the definition of exceptional circumstances can be vague and subject to interpretation. 

Consider the following two functions:

In [22]:
def divide_with_exception(dividend: int, divisor: int) -> None:
    try:
        print(f"{dividend / divisor=}")
    except ZeroDivisionError:
        print("You can't divide by zero")
def divide_with_if(dividend: int, divisor: int) -> None:
    if divisor == 0:
        print("You can't divide by zero")
    else:
        print(f"{dividend / divisor=}")


These two functions behave identically. 

If divisor is zero, an error message is printed; otherwise, a message printing the result of the division is displayed. 

We could avoid ZeroDivisionError ever being thrown by testing for it with an if statement. 

In this example, the test for a valid division is relatively simple-looking (divisor == 0). 

In some cases, it can be rather complex. 

In some cases, it may involve computing intermediate results. 

In the worst cases, the test for "will this work?" involves using a number of other methods of a class to – in effect – dry-run the operation to see if there would be an error along the way.

Therefore, it is wise to use exceptions for exceptional circumstances, even if those circumstances are only a little bit exceptional. 

Taking this argument further, exception syntax can be effective for flow control. 

Like an if statement, exceptions can be used for decision making, branching, and message passing.

Imagine an inventory application for a company that sells widgets and gadgets. 

When a customer makes a purchase, the item can either be available, in which case the item is removed from inventory and the number of items left is returned, or it might be out of stock. 

Now, being out of stock is a perfectly normal thing to happen in an inventory application. 

It is certainly not an exceptional circumstance. 

But what do we return if it's out of stock? A string saying "out of stock"? 

A negative number? In both cases, the calling method would have to check whether the return value is a positive integer or something else, to determine if it is out of stock. 

That seems a bit messy, especially if we forget to do it somewhere in our code.

Instead, we can raise an OutOfStock exception and use the try statement to direct program flow control. Make sense? 

In addition, we want to make sure we don't sell the same item to two different customers, or sell an item that isn't in stock yet. 

One way to facilitate this is to lock each type of item to ensure only one person can update it at a time. 

The user must lock the item, manipulate the item (purchase, add stock, count items left...), and then unlock the item. (This is, in effect, a context manager, one subject of Chapter 8.)

Here's an incomplete Inventory example with docstrings that describes what some of the methods should do:

In [23]:
class OutOfStock(Exception):
    pass
class InvalidItemType(Exception):
    pass
class Inventory:
    def __init__(self, stock: list[ItemType]) -> None:
        pass
    def lock(self, item_type: ItemType) -> None:
        """Context Entry.
        Lock the item type so nobody else can manipulate the
        inventory while we're working."""
        pass
    def unlock(self, item_type: ItemType) -> None:
        """Context Exit.
        Unlock the item type."""
        pass
    def purchase(self, item_type: ItemType) -> int:
        """If the item is not locked, raise a
        ValueError because something went wrong.
        If the item_type does not exist,
          raise InvalidItemType.
        If the item is currently out of stock,
          raise OutOfStock.
        If the item is available,
          subtract one item; return the number of items left.
        """
        # Mocked results.
        if item_type.name == "Widget":
            raise OutOfStock(item_type)
        elif item_type.name == "Gadget":
            return 42
        else:
            raise InvalidItemType(item_type)


NameError: name 'ItemType' is not defined

We could hand this object prototype to a developer and have them implement the methods to do exactly as they say while we work on the code needed to make a purchase. 

We'll use Python's robust exception handling to consider different branches, depending on how the purchase was made. 

We can even write a test case to be sure there's no question about how this class should work.

Here's a definition of ItemType, just to round out the example:

In [28]:
class ItemType:
    def __init__(self, name: str) -> None:
        self.name = name
        self.on_hand = 0

class OutOfStock(Exception):
    pass
class InvalidItemType(Exception):
    pass
class Inventory:
    def __init__(self, stock: list[ItemType]) -> None:
        pass
    def lock(self, item_type: ItemType) -> None:
        """Context Entry.
        Lock the item type so nobody else can manipulate the
        inventory while we're working."""
        pass
    def unlock(self, item_type: ItemType) -> None:
        """Context Exit.
        Unlock the item type."""
        pass
    def purchase(self, item_type: ItemType) -> int:
        """If the item is not locked, raise a
        ValueError because something went wrong.
        If the item_type does not exist,
          raise InvalidItemType.
        If the item is currently out of stock,
          raise OutOfStock.
        If the item is available,
          subtract one item; return the number of items left.
        """
        # Mocked results.
        if item_type.name == "Widget":
            raise OutOfStock(item_type)
        elif item_type.name == "Gadget":
            return 42
        else:
            raise InvalidItemType(item_type)


In [29]:
widget = ItemType("Widget")
gadget = ItemType("Gadget")
inv = Inventory([widget, gadget])
item_to_buy = widget
inv.lock(item_to_buy)
try:
    num_left = inv.purchase(item_to_buy)
except InvalidItemType:
    print(f"Sorry, we don't sell {item_to_buy.name}")
except OutOfStock:
    print("Sorry, that item is out of stock.")
else:
    print(f"Purchase complete. There are {num_left} {item_to_buy.name}s left")
finally:
    inv.unlock(item_to_buy)



Sorry, that item is out of stock.


All the possible exception handling clauses are used to ensure the correct actions happen at the correct time. 

Even though OutOfStock is not a terribly exceptional circumstance, we are able to use an exception to handle it suitably. 

This same code could be written with an if...elif...else structure, but it wouldn't be as easy to read or maintain.

As an aside, one of the exception messages, There are {num_left} {item_to_buy.name}s left, suffers from a goofy English grammar problem. 

When there's only one item left, it needs a major revision to There is {num_left} {item_to_buy.name} left. 

In order to support a sensible approach to translation, it's best to avoid fiddling around with grammar details inside the f-string. 

It's best to deal with it in the else: clause, using something like this to select the message with appropriate grammar:

In [30]:
msg = (
    f"there is {num_left} {item_to_buy.name} left" 
    if num_left == 1 
    else f"there are {num_left} {item_to_buy.name}s left")
print(msg)


NameError: name 'num_left' is not defined

We can also use exceptions to pass messages between different methods. 

For example, if we wanted to inform the customer as to what date the item is expected to be in stock again, we could ensure our OutOfStock object requires a back_in_stock parameter when it is constructed. 

Then, when we handle the exception, we can check that value and provide additional information to the customer. 

The information attached to the object can be easily passed between two different parts of the program. 

The exception could even provide a method that instructs the inventory object to reorder or backorder an item.

Using exceptions for flow control can make for some handy program designs. 

The important thing to take from this discussion is that exceptions are not a bad thing that we should try to avoid. 

Having an exception occur does not mean that you should have prevented this exceptional circumstance from happening. 

Rather, it is just a powerful way to communicate information between two sections of code that may not be directly calling each other.